In [ ]:
import whisperx

# identify word


In [ ]:
device = "cpu"
audio_file = "input.mp4"
batch_size = 32  # reduce if low on GPU mem
compute_type = "float32"  # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"])  # before alignment

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(
    language_code=result["language"], device=device
)
result = whisperx.align(
    result["segments"], model_a, metadata, audio, device, return_char_alignments=False
)

In [ ]:
result.keys()

In [ ]:
result

In [ ]:
result["segments"]

# write identify word to srt file


In [ ]:
from datetime import timedelta
import os


def write_segments_to_srt(segments):
    for index, segment in enumerate(segments):
        startTime = str(0) + str(timedelta(seconds=int(segment["start"]))) + ",000"
        endTime = str(0) + str(timedelta(seconds=int(segment["end"]))) + ",000"
        text = segment["text"]
        print(text)
        segment = f"{index + 1}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

        srtFilename = os.path.join(f"subtitles.srt")
        with open(srtFilename, "a", encoding="utf-8") as srtFile:
            srtFile.write(segment)


write_segments_to_srt(result["segments"])